In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pytorch_tabnet.pretraining import TabNetPretrainer
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import numpy as np


In [2]:
df_train = pd.DataFrame(np.random.rand(700, 42), columns=[f"feature_{i}" for i in range(42)])
df_test = pd.DataFrame(np.random.rand(300, 42), columns=[f"feature_{i}" for i in range(42)])

y_train = (np.random.rand(700) > 0.5).astype(np.float32)
y_test = (np.random.rand(300) > 0.5).astype(np.float32)


scaler = MinMaxScaler()
X_train = scaler.fit_transform(df_train.values)
X_test = scaler.transform(df_test.values)

# y_train = y_train.values.astype(np.float32)
# y_test = y_test.values.astype(np.float32)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


torch.Size([700, 42]) torch.Size([700]) torch.Size([300, 42]) torch.Size([300])


In [3]:
y_train

tensor([0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0.,
        0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
        1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
        0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1.,
        0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0.,
        1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
        0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
        1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
        0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1.,
        1., 0., 0., 1., 1., 0., 0., 1., 

In [4]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [5]:
tabnet_params = {
    "n_d": 16,
    "n_a": 16,
    "n_steps": 3,
    "n_shared": 2,
    "n_independent": 2,
    "gamma": 1.3,
    "epsilon": 1e-15,
    "momentum": 0.98,
    "mask_type": "sparsemax",
    "lambda_sparse": 1e-3
}


unsupervised_model = TabNetPretrainer(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    **tabnet_params
)
 

unsupervised_model.fit(
    X_train,
    eval_set=[X_test],  
    pretraining_ratio=0.8,
    max_epochs=101,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

e:\User\PC\ads_test\.conda\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 63.75671| val_0_unsup_loss_numpy: 10.578660011291504|  0:00:00s
epoch 1  | loss: 42.51789| val_0_unsup_loss_numpy: 8.89208984375|  0:00:00s
epoch 2  | loss: 31.41717| val_0_unsup_loss_numpy: 6.278289794921875|  0:00:00s
epoch 3  | loss: 23.50925| val_0_unsup_loss_numpy: 5.46442985534668|  0:00:00s
epoch 4  | loss: 18.19841| val_0_unsup_loss_numpy: 4.289790153503418|  0:00:00s
epoch 5  | loss: 14.19336| val_0_unsup_loss_numpy: 3.9120900630950928|  0:00:00s
epoch 6  | loss: 11.25805| val_0_unsup_loss_numpy: 3.352250099182129|  0:00:00s
epoch 7  | loss: 8.54979 | val_0_unsup_loss_numpy: 2.9933600425720215|  0:00:00s
epoch 8  | loss: 6.96652 | val_0_unsup_loss_numpy: 2.5284500122070312|  0:00:01s
epoch 9  | loss: 5.79663 | val_0_unsup_loss_numpy: 2.4295599460601807|  0:00:01s
epoch 10 | loss: 4.60288 | val_0_unsup_loss_numpy: 1.9948400259017944|  0:00:01s
epoch 11 | loss: 3.824   | val_0_unsup_loss_numpy: 1.7746299505233765|  0:00:01s
epoch 12 | loss: 3.05416 | val_0_unsup

e:\User\PC\ads_test\.conda\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [6]:
# Truy cập vào mô hình TabNet bên trong
from torchinfo import summary

tabnet_model = unsupervised_model.network

summary(tabnet_model, input_size=X_train.shape) 

Layer (type:depth-idx)                                       Output Shape              Param #
TabNetPretraining                                            [700, 42]                 --
├─EmbeddingGenerator: 1-1                                    [700, 42]                 --
├─TabNetEncoder: 1-2                                         [700, 16]                 --
│    └─BatchNorm1d: 2-1                                      [700, 42]                 84
│    └─FeatTransformer: 2-2                                  [700, 32]                 4,352
│    │    └─GLU_Block: 3-1                                   [700, 32]                 4,992
│    └─ModuleList: 2-12                                      --                        (recursive)
│    │    └─FeatTransformer: 3-17                            --                        (recursive)
│    └─FeatTransformer: 2-6                                  --                        (recursive)
│    │    └─GLU_Block: 3-5                                   -

In [7]:
encoder = tabnet_model.encoder

print("\nEncoder Summary:")
print(encoder)




Encoder Summary:
TabNetEncoder(
  (initial_bn): BatchNorm1d(42, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  (initial_splitter): FeatTransformer(
    (shared): GLU_Block(
      (shared_layers): ModuleList(
        (0): Linear(in_features=42, out_features=64, bias=False)
        (1): Linear(in_features=32, out_features=64, bias=False)
      )
      (glu_layers): ModuleList(
        (0): GLU_Layer(
          (fc): Linear(in_features=42, out_features=64, bias=False)
          (bn): GBN(
            (bn): BatchNorm1d(64, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
          )
        )
        (1): GLU_Layer(
          (fc): Linear(in_features=32, out_features=64, bias=False)
          (bn): GBN(
            (bn): BatchNorm1d(64, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
          )
        )
      )
    )
    (specifics): GLU_Block(
      (glu_layers): ModuleList(
        (0-1): 2 x GLU_Layer(
          (fc): Linear(in_fea

In [8]:
decoder = tabnet_model.decoder

print("\nDecoder Summary:")
print(decoder)


Decoder Summary:
TabNetDecoder(
  (feat_transformers): ModuleList(
    (0-2): 3 x FeatTransformer(
      (shared): GLU_Block(
        (shared_layers): ModuleList(
          (0): Linear(in_features=16, out_features=32, bias=False)
        )
        (glu_layers): ModuleList(
          (0): GLU_Layer(
            (fc): Linear(in_features=16, out_features=32, bias=False)
            (bn): GBN(
              (bn): BatchNorm1d(32, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
            )
          )
        )
      )
      (specifics): GLU_Block(
        (glu_layers): ModuleList(
          (0): GLU_Layer(
            (fc): Linear(in_features=16, out_features=32, bias=False)
            (bn): GBN(
              (bn): BatchNorm1d(32, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
            )
          )
        )
      )
    )
  )
  (reconstruction_layer): Linear(in_features=16, out_features=42, bias=False)
)


In [9]:
sample_input = torch.tensor(X_train[:5])  

try:
    result = tabnet_model.encoder(sample_input)
    if isinstance(result, tuple):
        print(f'TabNet encoder trả về {len(result)} giá trị.')
        for i, res in enumerate(result):
            print(f'Giá trị {i + 1} shape: {res.shape}')
    else:
        print('TabNet encoder chỉ trả về một giá trị.')
        print(f'Giá trị shape: {result.shape}')
except Exception as e:
    print(f'Đã xảy ra lỗi: {e}')

TabNet encoder trả về 2 giá trị.
Đã xảy ra lỗi: 'list' object has no attribute 'shape'


C:\Users\PC\AppData\Local\Temp\ipykernel_28040\992752778.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample_input = torch.tensor(X_train[:5])


In [10]:
class Sampling(nn.Module):
    def __init__(self, seed=1337):
        super(Sampling, self).__init__()
        self.seed = seed

    def forward(self, inputs):
        z_mean, z_log_var = inputs
        batch = z_mean.size(0)
        dim = z_mean.size(1)
        # print(batch, dim)
        epsilon = torch.randn(batch, dim, generator=torch.Generator().manual_seed(self.seed))
        return z_mean + torch.exp(0.5 * z_log_var) * epsilon

In [11]:
class VAE_Encoder(nn.Module):
    def __init__(self, latent_dim):
        super(VAE_Encoder, self).__init__()
        self.tabnet_encoder = tabnet_model.encoder
        self.mlp = nn.Sequential(
            nn.Linear(16, 128),  
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 96),
            nn.ReLU(),
            nn.Linear(96, latent_dim)
        )
        self.fc_mean = nn.Linear(latent_dim, latent_dim)
        self.fc_log_var = nn.Linear(latent_dim, latent_dim)
        self.sampling = Sampling()

    def forward(self, x):
        steps_output, _ = self.tabnet_encoder(x)
        encoded = steps_output[-1]
        # print("Shape of encoded tensor:", encoded.shape)
        encoded = self.mlp(encoded)
        z_mean = self.fc_mean(encoded)
        z_log_var = self.fc_log_var(encoded)
        z = self.sampling((z_mean, z_log_var))
        # print(f'Shape of z: {z.shape} - {z_log_var.shape} -{z_log_var.shape}')
        return z_mean, z_log_var, z


In [12]:
class VAE_Decoder(nn.Module):
    def __init__(self, latent_dim,encoded_dim, output_dim):
        super(VAE_Decoder, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(latent_dim, 32),   
            nn.ReLU(),
            nn.Linear(32, 96),
            nn.ReLU(),
            nn.Linear(96, 96),
            nn.ReLU(),
            nn.Linear(96, encoded_dim),  
        )
        self.tabnet_decoder = tabnet_model.decoder
        self.reshape = nn.Unflatten(1, (encoded_dim,))
        self.output_dim=output_dim


    def forward(self, z):
        x = F.relu(self.mlp(z))

        # print("Shape before reshape:", x.shape)
        # x = self.reshape(x)
        x = x[None, ...]

        # print("Shape after reshape:", x.shape)
        # x = x.view(x.size(0), output_dim)
        
        output = self.tabnet_decoder(x)
        # print(output.shape)
        # print("Shape of output from tabnet_decoder:", output.shape)
        output = torch.sigmoid(output)
        output = output.view(-1, self.output_dim)
        return output

In [13]:
def check_data_range(tensor, name):
    if not torch.all((tensor >= 0) & (tensor <= 1)):
        print(f"{name} contains values outside the range [0, 1]")
        print(f"{name} min: {tensor.min()}, max: {tensor.max()}")

In [14]:
class VAE(nn.Module):
    def __init__(self, encoder, decoder):
        super(VAE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x):
        z_mean, z_log_var, z = self.encoder(x)
        # print("Shape after encoder:", z.shape)

        reconstruction = self.decoder(z)
        # print("Shape after decoder:", reconstruction.shape)

        return reconstruction, z_mean, z_log_var

    def train_step(self, data, optimizer):
        optimizer.zero_grad()
        # z_mean, z_log_var, z = self.encoder(data)
        # reconstruction = self.decoder(z)
        reconstruction, z_mean, z_log_var = self.forward(data)
        # print(check_data_range(data, 'data'))
        # print(check_data_range(reconstruction, 'reconstruction'))
        # reconstruction_loss = torch.mean(
        #     torch.sum(
        #         F.binary_cross_entropy(reconstruction, data, reduction='none'),
        #         dim=1
        #     )
        # )
        reconstruction_loss = torch.mean(
            torch.sum(
                F.binary_cross_entropy_with_logits(reconstruction, data, reduction='none'),
                dim=1
                # dim=(1, 2)
                )  # Tính tổng trên chiều (1, 2)
        )
        # print(f'Shape reconstruction_loss: {reconstruction_loss.shape}')

        kl_loss = -0.5 * torch.sum(1 + z_log_var - z_mean.pow(2) - z_log_var.exp(), dim=1)

        # print(f'Shape z_mean: {z_mean.shape}')
        # print(f'Shape z_log_var: {z_log_var.shape}')
        kl_loss = torch.mean(torch.sum(kl_loss))
        # kl_loss = torch.mean(torch.sum(kl_loss, dim=1))
        total_loss = reconstruction_loss + kl_loss
        # print(f'Shape k1 loss: {kl_loss.shape}')
        total_loss.backward()
        optimizer.step()
        return total_loss.item(), reconstruction_loss.item(), kl_loss.item()

In [15]:
latent_dim = 64
encoded_dim = 16
output_dim = X_train.shape[1]
input_dim = X_train.shape[1]
print(input_dim)


42


In [16]:
vae_encoder = VAE_Encoder(latent_dim=latent_dim)
print("Encoder Summary:")
summary(vae_encoder, input_size=(700, input_dim))

Encoder Summary:


Layer (type:depth-idx)                                       Output Shape              Param #
VAE_Encoder                                                  [700, 64]                 --
├─TabNetEncoder: 1-1                                         [700, 16]                 --
│    └─BatchNorm1d: 2-1                                      [700, 42]                 84
│    └─FeatTransformer: 2-2                                  [700, 32]                 4,352
│    │    └─GLU_Block: 3-1                                   [700, 32]                 4,992
│    └─ModuleList: 2-12                                      --                        (recursive)
│    │    └─FeatTransformer: 3-17                            --                        (recursive)
│    └─FeatTransformer: 2-6                                  --                        (recursive)
│    │    └─GLU_Block: 3-5                                   --                        (recursive)
│    └─ModuleList: 2-12                              

In [17]:
x = torch.randn(800, 42) 
steps_output, _ = tabnet_model.encoder(x)
encoded = steps_output[-1]
print(f"Encoded shape: {encoded.shape}")

Encoded shape: torch.Size([800, 16])


In [18]:
import torch

x = torch.randn(800, 42)  

steps_output, _ = tabnet_model.encoder(x)
print("Shape of encoder output:", [output.shape for output in steps_output])

decoder_input = steps_output[-1] 
decoder_input = decoder_input[None, ...]
try:
    decoder_output = tabnet_model.decoder(decoder_input)
    print(f"Decoder shape: {decoder_output.shape}")
except ValueError as e:
    print(f"Error: {e}")


Shape of encoder output: [torch.Size([800, 16]), torch.Size([800, 16]), torch.Size([800, 16])]
Decoder shape: torch.Size([800, 42])


In [19]:
# x = torch.randn(800, 16) 
# print(x.shape)
# steps_output, _ = tabnet_model.decoder(x)
# decoder = steps_output[-1]
# print(f"Decoder shape: {decoder.shape}")

In [20]:
vae_decoder = VAE_Decoder(latent_dim=latent_dim, encoded_dim=encoded_dim, output_dim=output_dim)
print("Decoder Summary:")
summary(vae_decoder, input_size=(700, latent_dim), device="cpu")

Decoder Summary:


Layer (type:depth-idx)                                       Output Shape              Param #
VAE_Decoder                                                  [700, 42]                 --
├─Sequential: 1-1                                            [700, 16]                 --
│    └─Linear: 2-1                                           [700, 32]                 2,080
│    └─ReLU: 2-2                                             [700, 32]                 --
│    └─Linear: 2-3                                           [700, 96]                 3,168
│    └─ReLU: 2-4                                             [700, 96]                 --
│    └─Linear: 2-5                                           [700, 96]                 9,312
│    └─ReLU: 2-6                                             [700, 96]                 --
│    └─Linear: 2-7                                           [700, 16]                 1,552
├─TabNetDecoder: 1-2                                         [700, 42]             

In [21]:
vae = VAE(encoder=vae_encoder, decoder=vae_decoder)
summary(vae, input_size=(700, input_dim), device='cpu')

Layer (type:depth-idx)                                            Output Shape              Param #
VAE                                                               [700, 42]                 --
├─VAE_Encoder: 1-1                                                [700, 64]                 --
│    └─TabNetEncoder: 2-1                                         [700, 16]                 --
│    │    └─BatchNorm1d: 3-1                                      [700, 42]                 84
│    │    └─FeatTransformer: 3-2                                  [700, 32]                 9,344
│    │    └─ModuleList: 3-12                                      --                        (recursive)
│    │    └─FeatTransformer: 3-6                                  --                        (recursive)
│    │    └─ModuleList: 3-12                                      --                        (recursive)
│    │    └─FeatTransformer: 3-6                                  --                        (recursive)
│    │

In [22]:
learning_rate=0.0001
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
num_epochs = 10
for epoch in range(num_epochs):
    vae.train()
    train_loss = 0
    for batch_data, _ in train_loader:
        # print(batch_data)
        loss, rec_loss, kl_loss = vae.train_step(batch_data, optimizer)
        train_loss += loss

    train_loss /= len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}")

Epoch 1/10, Loss: 39.1144
Epoch 2/10, Loss: 36.3957
Epoch 3/10, Loss: 34.5533
Epoch 4/10, Loss: 33.0103
Epoch 5/10, Loss: 31.7651
Epoch 6/10, Loss: 30.9912
Epoch 7/10, Loss: 30.6230
Epoch 8/10, Loss: 30.4859
Epoch 9/10, Loss: 30.4281
Epoch 10/10, Loss: 30.3930
